In [10]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Función para cargar y preprocesar imágenes
def load_and_preprocess_images(data, img_size=(96, 96), path_col='path', label_col='label', filter_labels=None):
    images = []
    labels = []
    for index, row in data.iterrows():
        if filter_labels is None or row[label_col] in filter_labels:
            img_path = row[path_col].replace('..', 'data').replace('\\', '/')
            img = load_img(img_path, target_size=img_size, color_mode='rgb')  # Cambiado a 'rgb'
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(row[label_col])
    images = np.array(images, dtype='float32') / 255.0
    return images, labels

In [31]:
def load_and_preprocess_images2(data, img_size=(96, 96)):
    images = []
    for index, row in data.iterrows():
        img_path = f"data/data/images/test/{row['id_img']}.jpg"
        img = load_img(img_path, target_size=img_size, color_mode='rgb')  # Cambiado a 'rgb'
        img_array = img_to_array(img)
        images.append(img_array)
    images = np.array(images, dtype='float32') / 255.0
    return images

In [15]:
# Cargar los datos
train_data = pd.read_csv('data/train_set.csv')
test_data = pd.read_csv('test_set.csv')
sample_submission = pd.read_csv('sample_submision.csv')

In [32]:
test_images = load_and_preprocess_images2(test_data)

In [16]:
# Ajustar los nombres de las columnas según sea necesario
path_col = 'path'  # Cambia esto si el nombre de la columna es diferente
label_col = 'label'

In [6]:
# Mostrar las columnas de los DataFrames
print("Columnas de train_set.csv:", train_data.columns)
print("Columnas de test_set.csv:", test_data.columns)

Columnas de train_set.csv: Index(['id_img', 'path', 'label'], dtype='object')
Columnas de test_set.csv: Index(['id_img'], dtype='object')


In [17]:
# Cargar y procesar imágenes de entrenamiento
train_images, train_labels = load_and_preprocess_images(train_data, path_col=path_col, label_col=label_col, filter_labels=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'])

In [18]:
# Codificar etiquetas para 7 clases
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_categorical = to_categorical(train_labels_encoded, num_classes=7)

In [20]:
# Usar ResNet50 para extraer características
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

In [21]:
# Extraer características de las imágenes
features = model.predict(train_images)

In [22]:
# Dividir los datos en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(features, train_labels_encoded, test_size=0.2, random_state=42)

In [38]:
# Configurar el Grid Search para Random Forest
param_grid = {
    'n_estimators': [3000, 4000],
    'max_depth': [None, 1, 5],
    'min_samples_split': [1, 2, 3],
    'min_samples_leaf': [1]
}
clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(clf, param_grid, cv=3, n_jobs=1, verbose=2)

In [39]:
# Entrenar el modelo usando Grid Search
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=3000; total time=   1.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=3000; total time=   0.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=3000; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=4000; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=4000; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=1, n_estimators=4000; total time=   0.0s


KeyboardInterrupt: 

In [27]:
# Obtener los mejores parámetros
best_params = grid_search.best_params_
print(f"Mejores Parámetros: {best_params}")

Mejores Parámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [41]:
rf.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=3, n_estimators=3000)

In [42]:
# rf = RandomForestClassifier(n_estimators=3000, max_depth=None, min_samples_split=3, min_samples_leaf=1)
val_predictions = rf.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)

In [45]:
print(f"Validation accuracy: {val_accuracy:.4f}")

Validation accuracy: 0.4241


In [43]:
# Evaluar el modelo con los mejores parámetros
best_clf = grid_search.best_estimator_
val_predictions = best_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation accuracy: {val_accuracy:.4f}")

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [33]:
# Extraer características de las imágenes de prueba
test_features = model.predict(test_images)

221/221 ━━━━━━━━━━━━━━━━━━━━ 211s 939ms/step


In [44]:
# Realizar predicciones sobre el conjunto de prueba
test_predictions = rf.predict(test_features)
test_predictions_decoded = label_encoder.inverse_transform(test_predictions)

In [34]:
# Realizar predicciones sobre el conjunto de prueba
test_predictions = best_clf.predict(test_features)
test_predictions_decoded = label_encoder.inverse_transform(test_predictions)

In [46]:
# Generar el archivo de submission con el nombre del modelo
submission_name = f"submission_random_forest_optimized.csv"
submission = sample_submission.copy()

In [48]:
# Asegurar que el submission mantenga el formato correcto
submission['label'] = test_predictions_decoded

submission.to_csv(f'data/submission/{submission_name}', index=False)

print(f"Submission file saved as {submission_name}")

Submission file saved as submission_random_forest_optimized.csv


In [49]:
submission

,id_img,label
0,10052,happy
1,10065,happy
2,10079,angry
3,10095,angry
4,10121,happy
...,...,...
7061,9806,surprise
7062,9830,neutral
7063,9853,happy
7064,9878,sad
